In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
TRADIER_API_KEY = "Jta3vCKbfk5HxsN8hHqEp4HlyPku"
headers = {'Authorization': "Bearer {}".format(TRADIER_API_KEY), 'Accept': 'application/json'}

In [35]:


def get_options_for_security(stock):
    response = requests.get('https://sandbox.tradier.com/v1/markets/options/lookup',
        params={'underlying': stock},
        headers=headers
    )

    json_response = response.json()
    return json_response['symbols'][0]['options']


def get_quotes_for_options(options,batch_size=25):
    chains = []
    for i in range(0, len(options),batch_size):
        options_batch = options[i:i+batch_size]
        response = requests.get('https://sandbox.tradier.com/v1/markets/quotes',
            params={'symbols': ','.join(options_batch), 'greeks': 'true'},
            headers=headers
        )    
        json_response = response.json()
        chains += json_response['quotes']['quote']
    return chains
    
def get_quote_for_security(stock):
    response = requests.get('https://sandbox.tradier.com/v1/markets/quotes',
        params={'symbols': stock, 'greeks': 'false'},
        headers=headers
    )    
    json_response = response.json()
    return json_response['quotes']['quote']
    

def get_most_shorted_stocks(df=True):
    table_cols = ['Symbol\nSymbol',
                 'Company Name',
                 'Price',
                 'Chg% (1D)',
                 'Chg% (YTD)',
                 'Short Interest',
                 'Short Date',
                 'Float',
                 'Float Shorted (%)']
    url = "https://www.marketwatch.com/tools/screener/short-interest"
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = soup.findAll('table')[0]
    si_cols =  [col.text.strip('\n') for col in si_table.find('thead').find('tr').findAll('th')]
    assert(si_cols == table_cols)
    rows = si_table.findAll('tr')[1:]
    data = []
    for row in rows: data.append([val.text.strip('\n') for val in row.findAll('td')])
    if df: return pd.DataFrame(data,columns=si_cols)
    return data


In [36]:
table = get_most_shorted_stocks()
print(table)

   Symbol\nSymbol                          Company Name    Price Chg% (1D)  \
0    PUBM\n\nPUBM                         PubMatic Inc.   $32.98    -2.54%   
1    WKHS\n\nWKHS                  Workhorse Group Inc.   $15.39     2.16%   
2    CLOV\n\nCLOV  Clover Health Investments Corp. Cl A   $15.33     6.87%   
3      FUV\n\nFUV                         Arcimoto Inc.   $14.16     2.68%   
4      GEO\n\nGEO                        GEO Group Inc.    $7.35     4.18%   
5    BLNK\n\nBLNK                    Blink Charging Co.   $39.08     1.45%   
6    OTRK\n\nOTRK                           Ontrak Inc.   $30.96    -3.37%   
7      WPG\n\nWPG           Washington Prime Group Inc.    $5.08     0.40%   
8    PETS\n\nPETS                   PetMed Express Inc.   $32.85     0.31%   
9    ESPR\n\nESPR            Esperion Therapeutics Inc.   $25.87     1.93%   
10   NKLA\n\nNKLA                          Nikola Corp.   $17.34     3.21%   
11   BBBY\n\nBBBY                Bed Bath & Beyond Inc.   $31.90